### This notebook uses a breathing actuation sequence that computes triggers inflation deflation based upon an average inhale/exhale duration and a scale factor

In [26]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures
import time
import threading
import numpy as np

In [27]:
def data_handler(unused_addr, args, data1, data2, data3, data4, data5, data6): #BITalino ServerBIT format
    global ecg_data, resp_data
    global osc_client
        
    B = args[0]
    HR = args[1]
    
    # BITalino ServerBIT
    ecg_data.append(float(np.random.rand(1)))
    resp_data.append(float(data6))
        
    if B.is_warmed_up:
        B.set_data(resp_data[-B.buffer_length:])
    
    if HR.is_warmed_up:
        HR.set_data(ecg_data[-HR.buffer_length:])


In [28]:
def warmup(B, resp_data, HR, ecg_data):
    """Function to launch once the system has warmed up.
    Sets the  data initially and launches a recursive update of features.
    Parameters
    ----------
    B: object containing breathing data and features
    resp_data: respiration signal data to set
    HR: object containing ECG data and features
    ecg_data: ecg signal data to set
    """
    
    if not B.is_warmed_up:
        print("Breathing WARMUP")
        B.set_data(resp_data[-B.buffer_length:])
        B.is_warmed_up = True
        B.update_loop()
        
    if not HR.is_warmed_up:
        print("ECG WARMUP")
        HR.set_data(ecg_data[-HR.buffer_length:])
        HR.is_warmed_up = True
        HR.update_loop()
        
    breathing_factor = 1.2
    
    timer_actuation1 = threading.Timer(0.1, async_breathing, ["1", "2", B, breathing_factor, True] )
    timer_actuation1.start()
    
    #timer_actuation2 = threading.Timer(0.1, breathing_intervals, ["2", 3, 3, True] )
    #timer_actuation2 = threading.Timer(0.1, pulsating, ["2", True] )
    #timer_actuation2.start()

In [29]:
def warmup_inflate(act_id, duration):
    global osc_client
    
    osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
    time.sleep(duration)
    osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)


In [30]:
def deflate(act_id, duration):
    global osc_client
    
    osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
    time.sleep(duration)
    osc_client.send_message("/actuator/" + act_id + "/inflate", 0.0)


### Different actuation sequences

In [31]:
# computes inhale and exhale interval duration as average duration of breath * breathing_factor 

def breathing_biofeedback(act_id, B, breathing_factor, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, breathing_biofeedback, [act_id, B, breathing_factor, False])
        timer_exhale.start()

    else:
        print("exhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(B.features['avg_exhale'] * breathing_factor, breathing_biofeedback, [act_id, B, breathing_factor, True])
        timer_inhale.start()

In [32]:
# specify inhale and exhale interval duration in seconds

def breathing_intervals(act_id, inhale_duration, exhale_duration, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        print("inhale: ", inhale_duration)
        osc_client.send_message("/actuator/" + act_id + "/inflate", 80.0)
        timer_exhale = threading.Timer(inhale_duration, breathing_intervals, [act_id, inhale_duration, exhale_duration, False])
        timer_exhale.start()

    else:
        print("exhale: ", exhale_duration)
        osc_client.send_message("/actuator/" + act_id + "/inflate", -80.0)
        timer_inhale = threading.Timer(exhale_duration, breathing_intervals, [act_id, inhale_duration, exhale_duration, True])
        timer_inhale.start()

In [33]:
def pulsating(act_id,inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 50.0)
        timer_deflate = threading.Timer(3, pulsating, [act_id, False])
        timer_deflate.start()

    else:
        osc_client.send_message("/actuator/" + act_id + "/inflate", -50.0)
        timer_inflate = threading.Timer(3, pulsating, [act_id, True])
        timer_inflate.start()

In [34]:
from random import uniform

def random_inflation(act_id, lower, upper, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    int_length = uniform(lower,upper)
    
    if inflate:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        timer_deflate = threading.Timer(int_length, random_inflation, [act_id, lower, upper, False])
        timer_deflate.start()

    else:
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inflate = threading.Timer(int_length, random_inflation, [act_id, lower, upper, True])
        timer_inflate.start()

In [35]:
def async_intervals(act_first, act_snd, inhale_dur, exhale_dur, inflate_first):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate_first:
        print("inhale: ", inhale_dur)
        osc_client.send_message("/actuator/" + act_first + "/inflate", 100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", -100.0)
        timer_inhale = threading.Timer(inhale_dur, async_intervals, [act_first, act_snd, inhale_dur, exhale_dur, False])
        timer_inhale.start()

    else:
        print("exhale: ", exhale_dur)
        osc_client.send_message("/actuator/" + act_first + "/inflate", -100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 100.0)
        timer_exhale = threading.Timer(exhale_dur, async_intervals, [act_first, act_snd, inhale_dur, exhale_dur, True])
        timer_exhale.start()

In [36]:
def async_breathing(act_first, act_snd, B, breathing_factor, inflate):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    if inflate:
        print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/" + act_first + "/inflate", 100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", -100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, async_breathing, [act_first, act_snd, B, breathing_factor, False])
        timer_exhale.start()

    else:
        print("exhale: ", B.features['avg_exhale'])
        osc_client.send_message("/actuator/" + act_first + "/inflate", -100.0)
        osc_client.send_message("/actuator/" + act_snd + "/inflate", 100.0)
        timer_inhale = threading.Timer(B.features['avg_exhale'] * breathing_factor, async_breathing, [act_first, act_snd, B, breathing_factor, True])
        timer_inhale.start()

In [37]:
# computes degree of inflation/deflation based on ECG features

# TODO: how to compute the inflation/deflation value?

def heartrate_actuation(act_id, HR, timer_interval):
    global osc_client, actuation_flag
    
    if not actuation_flag:
        return
    
    current_trend = HR.current_trends["hr_mean"]
    #current_trend = HR.current_trends["rmssd"]
    #current_trend = HR.current_trends["LF/HF ratio"]
    
    current_feature = HR.features[-1]["hr_mean"]
    #current_feature = HR.features[-1]["rmssd"]
    #current_feature = HR.features[-1]["LF/HF ratio"]
    
    if current_trend > 0:
        osc_client.send_message("/actuator/" + act_id + "/inflate", 100.0)
        #osc_client.send_message("/actuator/" + act_id + "/inflate", current_trend * 1000/current_feature)
        timer_exhale = threading.Timer(timer_interval, heartrate_actuation, [act_id, HR, timer_interval])
        timer_exhale.start()

    else:
        #osc_client.send_message("/actuator/inflate", current_trend * 1000/current_feature)
        osc_client.send_message("/actuator/" + act_id + "/inflate", -100.0)
        timer_inhale = threading.Timer(timer_interval, heartrate_actuation, [act_id, HR, timer_interval])
        timer_inhale.start()

#### Set up configuration: IP (BITalino, R-IoT or Node-RED), port and OSC address pattern

In [38]:
# Definitions
bitalino_ip = '192.168.0.100'
bitalino_port = 8000

riot_ip = "192.168.0.102"
riot_port = 31000 

# riot_ip = "192.168.1.40"
# riot_port = 12000 

#actuator_ip = '192.168.0.110'
actuator_ip = '192.168.0.100'
actuator_port = 32000

riot_address = "/raw/"
bitalino_address = "/0/bitalino"


# Configuration 
config_address = bitalino_address
config_ip = bitalino_ip
config_port = bitalino_port

osc_client = SimpleUDPClient(actuator_ip, actuator_port) 

actuation_flag = True

ecg_data = []
resp_data = []

B = biofeatures.breathing(data = np.ones(10), buffer_length=2000, srate=100)
HRV = biofeatures.hrv(data = np.ones(10), buffer_length=2000, srate=100)

my_dispatcher = dispatcher.Dispatcher()
my_dispatcher.map(config_address, data_handler, B, HRV)

server = osc_server.ThreadingOSCUDPServer((config_ip, config_port), my_dispatcher)
print("Serving on {}".format(server.server_address))

# Warmup
warmup_t = 10
timer_warmup = threading.Timer(warmup_t, warmup, [B,resp_data,HRV,ecg_data])
timer_warmup.start()

try: 
    server.serve_forever()
except KeyboardInterrupt:
    B.update_data_flag = False
    HRV.update_data_flag = False
    actuation_flag = False
    server.server_close()
    osc_client.send_message("/actuator/1/inflate", 0.0)
    osc_client.send_message("/actuator/2/inflate", 0.0)
except:
    raise

Serving on ('192.168.0.100', 8000)
Breathing WARMUP
{'breath_avg_len': 3.33, 'inhale_duration': 4.6, 'avg_inhale': 1.53, 'exhale_duration': 5.39, 'avg_exhale': 1.8, 'inhale_exhale_ratio': 0.85}
ECG WARMUP
[{'nni_mean': 470.0, 'hr_mean': 136.76, 'hr_std': 36.93, 'rmssd': 148.3, 'lf': 88.53, 'hf': 8573.56, 'LF/HF ratio': 0.01}]
inhale:  1.53
{'breath_avg_len': 3.52, 'inhale_duration': 4.76, 'avg_inhale': 1.59, 'exhale_duration': 5.81, 'avg_exhale': 1.94, 'inhale_exhale_ratio': 0.82}
[{'nni_mean': 470.0, 'hr_mean': 136.76, 'hr_std': 36.93, 'rmssd': 148.3, 'lf': 88.53, 'hf': 8573.56, 'LF/HF ratio': 0.01}, {'nni_mean': 483.33, 'hr_mean': 133.86, 'hr_std': 37.88, 'rmssd': 144.39, 'lf': 652.08, 'hf': 7963.65, 'LF/HF ratio': 0.08}]
{'breath_avg_len': 2.76, 'inhale_duration': 5.32, 'avg_inhale': 1.33, 'exhale_duration': 5.71, 'avg_exhale': 1.9, 'inhale_exhale_ratio': 0.93}
[{'nni_mean': 470.0, 'hr_mean': 136.76, 'hr_std': 36.93, 'rmssd': 148.3, 'lf': 88.53, 'hf': 8573.56, 'LF/HF ratio': 0.01}, 

exhale:  1.94
{'breath_avg_len': 3.3, 'inhale_duration': 8.63, 'avg_inhale': 1.73, 'exhale_duration': 7.85, 'avg_exhale': 1.96, 'inhale_exhale_ratio': 1.1}
[{'nni_mean': 488.52, 'hr_mean': 138.46, 'hr_std': 50.56, 'rmssd': 230.76, 'lf': 2549.41, 'hf': 10744.77, 'LF/HF ratio': 0.24}, {'nni_mean': -12.16, 'hr_mean': 146.24, 'hr_std': 61.87, 'rmssd': 2677.88, 'lf': 49532.1, 'hf': 21677.44, 'LF/HF ratio': 2.28}, {'nni_mean': 468.33, 'hr_mean': 146.19, 'hr_std': 54.59, 'rmssd': 229.28, 'lf': 2631.44, 'hf': 12065.46, 'LF/HF ratio': 0.22}, {'nni_mean': 468.71, 'hr_mean': 145.51, 'hr_std': 53.81, 'rmssd': 228.98, 'lf': 2769.65, 'hf': 13218.51, 'LF/HF ratio': 0.21}, {'nni_mean': 466.88, 'hr_mean': 145.53, 'hr_std': 52.94, 'rmssd': 225.6, 'lf': 2433.25, 'hf': 14382.54, 'LF/HF ratio': 0.17}]
{'breath_avg_len': 2.82, 'inhale_duration': 8.79, 'avg_inhale': 1.46, 'exhale_duration': 8.15, 'avg_exhale': 1.63, 'inhale_exhale_ratio': 1.08}
[{'nni_mean': -12.16, 'hr_mean': 146.24, 'hr_std': 61.87, 'rmssd

{'breath_avg_len': 4.0, 'inhale_duration': 10.35, 'avg_inhale': 2.07, 'exhale_duration': 9.63, 'avg_exhale': 1.93, 'inhale_exhale_ratio': 1.07}
[{'nni_mean': -11.5, 'hr_mean': 162.15, 'hr_std': 66.6, 'rmssd': 2967.39, 'lf': 40906.23, 'hf': 35535.5, 'LF/HF ratio': 1.15}, {'nni_mean': -22.64, 'hr_mean': 161.83, 'hr_std': 67.32, 'rmssd': 3027.97, 'lf': 41573.82, 'hf': 36748.55, 'LF/HF ratio': 1.13}, {'nni_mean': -12.12, 'hr_mean': 148.58, 'hr_std': 66.41, 'rmssd': 3175.86, 'lf': 41211.93, 'hf': 41492.63, 'LF/HF ratio': 0.99}, {'nni_mean': 574.52, 'hr_mean': 138.6, 'hr_std': 71.73, 'rmssd': 419.27, 'lf': 20735.91, 'hf': 70045.47, 'LF/HF ratio': 0.3}, {'nni_mean': -8.64, 'hr_mean': 158.68, 'hr_std': 68.04, 'rmssd': 3022.73, 'lf': 29275.29, 'hf': 38661.1, 'LF/HF ratio': 0.76}]
{'breath_avg_len': 4.0, 'inhale_duration': 9.98, 'avg_inhale': 2.0, 'exhale_duration': 10.0, 'avg_exhale': 2.0, 'inhale_exhale_ratio': 1.0}
[{'nni_mean': -22.64, 'hr_mean': 161.83, 'hr_std': 67.32, 'rmssd': 3027.97, 'l

inhale:  2.03
{'breath_avg_len': 4.0, 'inhale_duration': 9.4, 'avg_inhale': 2.35, 'exhale_duration': 10.58, 'avg_exhale': 2.12, 'inhale_exhale_ratio': 0.89}
[{'nni_mean': 484.47, 'hr_mean': 151.22, 'hr_std': 65.33, 'rmssd': 348.13, 'lf': 10526.09, 'hf': 26644.29, 'LF/HF ratio': 0.4}, {'nni_mean': 454.76, 'hr_mean': 156.16, 'hr_std': 62.29, 'rmssd': 288.33, 'lf': 5009.68, 'hf': 18657.72, 'LF/HF ratio': 0.27}, {'nni_mean': 485.79, 'hr_mean': 150.77, 'hr_std': 64.08, 'rmssd': 365.83, 'lf': 13874.26, 'hf': 29199.4, 'LF/HF ratio': 0.48}, {'nni_mean': -18.81, 'hr_mean': 156.71, 'hr_std': 67.99, 'rmssd': 3097.84, 'lf': 31617.62, 'hf': 29665.82, 'LF/HF ratio': 1.07}, {'nni_mean': -13.09, 'hr_mean': 163.04, 'hr_std': 69.39, 'rmssd': 3070.08, 'lf': 55544.51, 'hf': 37304.02, 'LF/HF ratio': 1.49}]
{'breath_avg_len': 3.33, 'inhale_duration': 10.39, 'avg_inhale': 1.73, 'exhale_duration': 9.59, 'avg_exhale': 1.92, 'inhale_exhale_ratio': 1.08}
[{'nni_mean': 454.76, 'hr_mean': 156.16, 'hr_std': 62.29, 

{'breath_avg_len': 3.33, 'inhale_duration': 10.38, 'avg_inhale': 2.08, 'exhale_duration': 9.6, 'avg_exhale': 1.92, 'inhale_exhale_ratio': 1.08}
[{'nni_mean': 422.17, 'hr_mean': 166.72, 'hr_std': 62.12, 'rmssd': 274.61, 'lf': 2492.31, 'hf': 15260.23, 'LF/HF ratio': 0.16}, {'nni_mean': -8.76, 'hr_mean': 171.74, 'hr_std': 68.35, 'rmssd': 2907.08, 'lf': 34886.25, 'hf': 25398.55, 'LF/HF ratio': 1.37}, {'nni_mean': -12.27, 'hr_mean': 171.37, 'hr_std': 68.59, 'rmssd': 3012.28, 'lf': 86610.77, 'hf': 43592.61, 'LF/HF ratio': 1.99}, {'nni_mean': 396.6, 'hr_mean': 174.11, 'hr_std': 60.7, 'rmssd': 248.16, 'lf': 2568.27, 'hf': 12169.91, 'LF/HF ratio': 0.21}, {'nni_mean': -9.37, 'hr_mean': 169.66, 'hr_std': 64.8, 'rmssd': 2915.65, 'lf': 49549.65, 'hf': 31874.46, 'LF/HF ratio': 1.55}]
{'breath_avg_len': 3.33, 'inhale_duration': 10.66, 'avg_inhale': 2.13, 'exhale_duration': 9.32, 'avg_exhale': 1.86, 'inhale_exhale_ratio': 1.14}
[{'nni_mean': -8.76, 'hr_mean': 171.74, 'hr_std': 68.35, 'rmssd': 2907.08,

{'breath_avg_len': 3.33, 'inhale_duration': 10.72, 'avg_inhale': 2.14, 'exhale_duration': 9.26, 'avg_exhale': 1.54, 'inhale_exhale_ratio': 1.16}
[{'nni_mean': -7.69, 'hr_mean': 166.93, 'hr_std': 68.76, 'rmssd': 2929.03, 'lf': 5085.67, 'hf': 15161.84, 'LF/HF ratio': 0.34}, {'nni_mean': -16.4, 'hr_mean': 158.83, 'hr_std': 72.46, 'rmssd': 3157.23, 'lf': 48498.43, 'hf': 37021.29, 'LF/HF ratio': 1.31}, {'nni_mean': -12.3, 'hr_mean': 158.84, 'hr_std': 71.08, 'rmssd': 3076.18, 'lf': 29461.91, 'hf': 28372.55, 'LF/HF ratio': 1.04}, {'nni_mean': 443.41, 'hr_mean': 163.26, 'hr_std': 65.53, 'rmssd': 330.69, 'lf': 10325.99, 'hf': 6996.13, 'LF/HF ratio': 1.48}, {'nni_mean': 409.57, 'hr_mean': 170.12, 'hr_std': 63.3, 'rmssd': 257.58, 'lf': 4419.35, 'hf': 5425.8, 'LF/HF ratio': 0.81}]
{'breath_avg_len': 4.0, 'inhale_duration': 10.11, 'avg_inhale': 2.02, 'exhale_duration': 9.87, 'avg_exhale': 1.97, 'inhale_exhale_ratio': 1.02}
exhale:  1.97
[{'nni_mean': -16.4, 'hr_mean': 158.83, 'hr_std': 72.46, 'rmss

exhale:  1.7
{'breath_avg_len': 4.0, 'inhale_duration': 9.53, 'avg_inhale': 1.91, 'exhale_duration': 10.45, 'avg_exhale': 2.09, 'inhale_exhale_ratio': 0.91}
[{'nni_mean': 410.43, 'hr_mean': 167.44, 'hr_std': 61.63, 'rmssd': 210.18, 'lf': 4772.85, 'hf': 5472.23, 'LF/HF ratio': 0.87}, {'nni_mean': 400.21, 'hr_mean': 171.11, 'hr_std': 61.16, 'rmssd': 208.65, 'lf': 4181.85, 'hf': 5192.09, 'LF/HF ratio': 0.81}, {'nni_mean': -11.41, 'hr_mean': 168.69, 'hr_std': 64.56, 'rmssd': 2879.41, 'lf': 30273.32, 'hf': 17129.65, 'LF/HF ratio': 1.77}, {'nni_mean': 408.94, 'hr_mean': 169.66, 'hr_std': 62.35, 'rmssd': 226.25, 'lf': 3391.41, 'hf': 5045.99, 'LF/HF ratio': 0.67}, {'nni_mean': 394.29, 'hr_mean': 172.6, 'hr_std': 59.93, 'rmssd': 204.26, 'lf': 3202.97, 'hf': 5068.06, 'LF/HF ratio': 0.63}]
{'breath_avg_len': 4.0, 'inhale_duration': 9.24, 'avg_inhale': 1.85, 'exhale_duration': 10.74, 'avg_exhale': 2.15, 'inhale_exhale_ratio': 0.86}
[{'nni_mean': 400.21, 'hr_mean': 171.11, 'hr_std': 61.16, 'rmssd':

{'breath_avg_len': 4.0, 'inhale_duration': 9.62, 'avg_inhale': 1.92, 'exhale_duration': 10.36, 'avg_exhale': 2.07, 'inhale_exhale_ratio': 0.93}
[{'nni_mean': 398.04, 'hr_mean': 172.44, 'hr_std': 61.43, 'rmssd': 214.08, 'lf': 1312.87, 'hf': 5613.3, 'LF/HF ratio': 0.23}, {'nni_mean': 401.67, 'hr_mean': 171.57, 'hr_std': 61.26, 'rmssd': 218.69, 'lf': 661.72, 'hf': 6530.81, 'LF/HF ratio': 0.1}, {'nni_mean': 461.75, 'hr_mean': 152.41, 'hr_std': 60.53, 'rmssd': 252.19, 'lf': 8349.23, 'hf': 9530.13, 'LF/HF ratio': 0.88}, {'nni_mean': 515.56, 'hr_mean': 143.09, 'hr_std': 64.83, 'rmssd': 362.3, 'lf': 5433.88, 'hf': 12326.9, 'LF/HF ratio': 0.44}, {'nni_mean': 464.0, 'hr_mean': 154.34, 'hr_std': 62.96, 'rmssd': 282.04, 'lf': 7396.31, 'hf': 9451.81, 'LF/HF ratio': 0.78}]
inhale:  1.92
{'breath_avg_len': 4.0, 'inhale_duration': 9.57, 'avg_inhale': 1.91, 'exhale_duration': 10.41, 'avg_exhale': 2.08, 'inhale_exhale_ratio': 0.92}
[{'nni_mean': 401.67, 'hr_mean': 171.57, 'hr_std': 61.26, 'rmssd': 218.6

{'breath_avg_len': 3.33, 'inhale_duration': 9.77, 'avg_inhale': 1.95, 'exhale_duration': 10.21, 'avg_exhale': 2.04, 'inhale_exhale_ratio': 0.96}
[{'nni_mean': 519.71, 'hr_mean': 144.53, 'hr_std': 66.72, 'rmssd': 396.71, 'lf': 18431.29, 'hf': 28998.51, 'LF/HF ratio': 0.64}, {'nni_mean': 512.22, 'hr_mean': 146.59, 'hr_std': 67.37, 'rmssd': 391.4, 'lf': 18717.38, 'hf': 31928.05, 'LF/HF ratio': 0.59}, {'nni_mean': -20.2, 'hr_mean': 160.09, 'hr_std': 67.44, 'rmssd': 3039.96, 'lf': 43086.74, 'hf': 21667.49, 'LF/HF ratio': 1.99}, {'nni_mean': 444.19, 'hr_mean': 158.58, 'hr_std': 62.09, 'rmssd': 278.89, 'lf': 6387.93, 'hf': 7025.83, 'LF/HF ratio': 0.91}, {'nni_mean': -8.02, 'hr_mean': 160.69, 'hr_std': 67.05, 'rmssd': 2969.28, 'lf': 13992.5, 'hf': 12487.07, 'LF/HF ratio': 1.12}]
{'breath_avg_len': 3.33, 'inhale_duration': 10.93, 'avg_inhale': 2.19, 'exhale_duration': 9.05, 'avg_exhale': 1.81, 'inhale_exhale_ratio': 1.21}
[{'nni_mean': 512.22, 'hr_mean': 146.59, 'hr_std': 67.37, 'rmssd': 391.4,

Exception in thread Thread-6181:
Traceback (most recent call last):
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/threading.py", line 1178, in run
    self.function(*self.args, **self.kwargs)
  File "/Users/annkatrin/Documents/Uni/Master/KTH/Master Project/biofeatures/notebooks/biofeatures.py", line 265, in update_loop
    self.update_timer.start()
  File "/Users/annkatrin/opt/anaconda3/lib/python3.7/threading.py", line 848, in start
    raise RuntimeError("threads can only be started once")
RuntimeError: threads can only be started once



In [40]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures
import time

actuator_ip = '192.168.0.100'
actuator_port = 32000

osc_client = SimpleUDPClient(actuator_ip, actuator_port) 
osc_client.send_message("/actuator/1/inflate", 0.0)
osc_client.send_message("/actuator/2/inflate", 0.0)



In [42]:
deflate("1", 5)
deflate("2", 5)

In [ ]:
warmup_inflate("1", 3)
warmup_inflate("2", 3)

In [ ]:
actuation_flag = True

pulsating("1", True)

In [ ]:
actuation_flag = False

In [ ]:
osc_client.send_message("/actuator/1/inflate", 0.0)

In [ ]:
print("hi")

In [21]:
deflate("2", 2)



In [20]:
deflate("1", 2)



In [22]:
B.features

{'breath_avg_len': 2.85,
 'inhale_duration': 10.58,
 'avg_inhale': 1.76,
 'exhale_duration': 9.4,
 'avg_exhale': 1.57,
 'inhale_exhale_ratio': 1.13}

In [23]:
actuation_flag = True

timer_actuation1 = threading.Timer(0.1, async_intervals, ["1", "2", 3, 3, True] )
timer_actuation1.start()



inhale:  3
exhale:  3
inhale:  3
exhale:  3
inhale:  3
exhale:  3
inhale:  3
exhale:  3
inhale:  3
exhale:  3
inhale:  3
exhale:  3
inhale:  3


In [24]:
actuation_flag = False